In [1]:
import os
import pandas as pd
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [3]:
def generate_df(directory: str, categories: dict) -> pd.DataFrame:
    data = []

    for category, label in categories.items():
        category_path = os.path.join(directory, category)
        for filename in os.listdir(category_path):
            file_path = os.path.join(category_path, filename)
            data.append([file_path, label])

    return pd.DataFrame(data, columns=['path', 'label'])

In [11]:
from google.colab import drive
drive.mount('/content/drive')

path_to_file = '/content/drive/My Drive/Brain/Brain MRI Images/Brain MRI Images'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
TRAIN_PATH = f'{path_to_file}/Train'
VALID_PATH = f'{path_to_file}/Validation'

CATEGORIES = {
    'Tumor': '1',
    'Normal': '0'
}

train_df = generate_df(TRAIN_PATH, CATEGORIES)
valid_df = generate_df(VALID_PATH, CATEGORIES)

In [19]:
train_df.head()

,path,label
0,/content/drive/My Drive/Brain/Brain MRI Images...,1
1,/content/drive/My Drive/Brain/Brain MRI Images...,1
2,/content/drive/My Drive/Brain/Brain MRI Images...,1
3,/content/drive/My Drive/Brain/Brain MRI Images...,1
4,/content/drive/My Drive/Brain/Brain MRI Images...,1


In [20]:
valid_df.head()

,path,label
0,/content/drive/My Drive/Brain/Brain MRI Images...,1
1,/content/drive/My Drive/Brain/Brain MRI Images...,1
2,/content/drive/My Drive/Brain/Brain MRI Images...,1
3,/content/drive/My Drive/Brain/Brain MRI Images...,1
4,/content/drive/My Drive/Brain/Brain MRI Images...,1


In [15]:
datagen = ImageDataGenerator(
    rescale=1./255,
)

In [21]:
train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=None,
    x_col="path",
    y_col="label",
    target_size=(224, 224),  # Resizing to match VGG16 input
    batch_size=32,
    class_mode='binary',
)

validation_generator = datagen.flow_from_dataframe(
    dataframe=valid_df,
    directory=None,
    x_col="path",
    y_col="label",
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 320 validated image filenames belonging to 2 classes.
Found 80 validated image filenames belonging to 2 classes.


In [22]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Create the model and add custom layers
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)

Epoch 1/10
10/10 [==============================] - 268s 26s/step - loss: 0.7981 - accuracy: 0.6094 - val_loss: 0.5460 - val_accuracy: 0.6406
Epoch 2/10
10/10 [==============================] - 243s 25s/step - loss: 0.4715 - accuracy: 0.7688 - val_loss: 0.7103 - val_accuracy: 0.6094
Epoch 3/10
10/10 [==============================] - 243s 25s/step - loss: 0.3825 - accuracy: 0.8000 - val_loss: 0.3594 - val_accuracy: 0.8594
Epoch 4/10
10/10 [==============================] - 253s 26s/step - loss: 0.2847 - accuracy: 0.8719 - val_loss: 0.2969 - val_accuracy: 0.8906
Epoch 5/10
10/10 [==============================] - 244s 25s/step - loss: 0.2032 - accuracy: 0.9438 - val_loss: 0.2624 - val_accuracy: 0.8750
Epoch 6/10
10/10 [==============================] - 283s 29s/step - loss: 0.1710 - accuracy: 0.9500 - val_loss: 0.2213 - val_accuracy: 0.9219
Epoch 7/10
10/10 [==============================] - 282s 29s/step - loss: 0.1555 - accuracy: 0.9469 - val_loss: 0.1731 - val_accuracy: 0.9531
Epoch 